In [51]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [52]:
seattle_file = r"C:\Users\njoku\OneDrive\Documents\GitHub\etlproject_2019\Resources\seattle_01.csv"
seattle_df = pd.read_csv(seattle_file, encoding="ISO-8859-1")
seattle_df.head()

,Unnamed: 0,room_id,host_id,room_type,address,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,last_modified,latitude,longitude,location,name,currency,rate_type
0,0,2318,2536,Entire home/apt,"Seattle, WA, United States",21,5.0,8,4,2.5,250,46:14.7,47.610819,-122.290816,0101000020E6100000D449B6BA9C925EC0416326512FCE...,"Casa Madrona - Urban Oasis, 1 block from the P...",USD,nightly
1,1,3335,4193,Entire home/apt,"Seattle, WA, United States",1,NaN,4,2,1.0,100,08:45.3,47.529846,-122.275840,0101000020E61000006FBBD05CA7915EC04DF564FED1C3...,Sweet Seattle Urban Homestead 2 Bdr,USD,nightly
2,2,4291,35749,Private room,"Seattle, WA, United States",63,4.5,2,1,1.0,82,04:19.9,47.687801,-122.313427,0101000020E6100000BDAB1E300F945EC0FB93F8DC09D8...,Sunrise in Seattle Master Suite,USD,nightly
3,3,5682,8993,Entire home/apt,"Seattle, WA, United States",462,5.0,2,0,1.0,49,11:25.5,47.523980,-122.359891,0101000020E6100000FCC7427408975EC009E1D1C611C3...,"Cozy Studio, min. to downtown -WiFi",USD,nightly
4,4,6606,14942,Entire home/apt,"Seattle, WA, United States",134,4.5,2,1,1.0,90,12:38.4,47.654109,-122.337605,0101000020E6100000D47D00529B955EC07782FDD7B9D3...,"Fab, private seattle urban cottage!",USD,nightly


In [53]:
texas_file = r"C:\Users\njoku\OneDrive\Documents\GitHub\etlproject_2019\Resources\Airbnb_Texas_Rentals.csv"
texas_df = pd.read_csv(texas_file, encoding="ISO-8859-1")
texas_df.head()

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May-16,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,Nov-10,"Stylish, fully remodeled home in upscale NW ?...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,3,$59,1,Houston,Jan-17,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...
3,4,$60,1,Bryan,Feb-16,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...
4,5,$75,2,Fort Worth,Feb-17,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...


### Transform premise DataFrame

In [54]:
# Create a filtered dataframe from specific columns
seattle_cols = ["price", "bedrooms", "address"]
seattle_transformed= seattle_df[seattle_cols].copy()

# Rename the column headers
seattle_transformed = seattle_transformed.rename(columns={"price": "Price",
                                                          "bedrooms": "Bedrooms",
                                                          "address": "City"})

# Clean the data by dropping duplicates and setting the index
seattle_transformed.set_index("Bedrooms", inplace=True)

seattle_transformed.head()

,Price,City
Bedrooms,,
4,250,"Seattle, WA, United States"
2,100,"Seattle, WA, United States"
1,82,"Seattle, WA, United States"
0,49,"Seattle, WA, United States"
1,90,"Seattle, WA, United States"


### Transform county DataFrame

In [55]:
texas_cols = ["average_rate_per_night", "bedrooms_count", "city"]
texas_transformed = texas_df[texas_cols].copy()

# Rename the column headers
texas_transformed = texas_transformed.rename(columns={"average_rate_per_night": "Price",
                                                         "bedrooms_count": "Bedrooms",
                                                         "city": "City",
                                                         })

# Set index
texas_transformed.set_index("Bedrooms", inplace=True)

texas_transformed.head()

,Price,City
Bedrooms,,
2,$27,Humble
4,$149,San Antonio
1,$59,Houston
1,$60,Bryan
2,$75,Fort Worth


### Create database connection

In [56]:
connection_string = "postgres:postgres@localhost:5432/airbnb"
engine = create_engine(f'postgresql://{connection_string}')

In [57]:
# Confirm tables
engine.table_names()

['airbnbwa', 'airbnbtx']

### Load DataFrames into database

In [58]:
seattle_transformed.to_sql(name='airbnbwa', con=engine, if_exists='append', index=True)

In [59]:
texas_transformed.to_sql(name='airbnbtx', con=engine, if_exists='append', index=True)